In [1]:
import os
import pandas as pd
import csv
import numpy as np
from stockstats import wrap

### Get data from files

In [5]:
# get files
def files(dir):
    names = []
    for root, dirs, files in os.walk(dir):  
        for file in files:  
            if os.path.splitext(file)[1] == '.csv':  # csv files
                names.append(file)
    return names

In [6]:
# names for all files
names = files(os.getcwd() + "/data")

### Data for Optimization

In [7]:
# short-term: 1 day
date = "2021-08-28"

with open("op_data_1.csv",'a', newline = "") as f:
    csv_write = csv.writer(f)
    csv_write.writerow(["Name", "Open", "Close"])

    for file in names:
        content = pd.read_csv("data/"+ file)
        name = file.split(".")[0]
        open_price = content[content['Date'] == date].to_numpy()[0,1]
        close_price = content[content['Date'] == date].to_numpy()[0,4]
        csv_write.writerow([name, open_price, close_price])

In [8]:
# long-term: 1 month
start_date = "2021-07-28"
end_date = "2021-08-28"

with open("op_data_2.csv",'a', newline = "") as f:
    csv_write = csv.writer(f)
    csv_write.writerow(["Name", "Open", "Close", "Risk"])

    for file in names:
        content = pd.read_csv("data/"+ file)
        name = file.split(".")[0]
        open_price = content[content['Date'] == start_date].to_numpy()[0,1]
        close_price = content[content['Date'] == end_date].to_numpy()[0,4]
        
        # calculate variance
        close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)].to_numpy()[:,4]
        risk = np.var(close, ddof = 1)
        
        csv_write.writerow([name, open_price, close_price, risk])

In [9]:
# 1 month time series
start_date = "2021-07-28"
end_date = "2021-08-28"

# initialize
content = pd.read_csv("data/"+ names[0])
name = names[0].split(".")[0]

close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)][["Date", "Close"]]
total = close
total.rename(columns = {'Close': name}, inplace = True)

for file in names[1:]:
    content = pd.read_csv("data/"+ file)
    name = file.split(".")[0]

    close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)].to_numpy()[:,4]
    total[name] = close

    total.to_csv("op_data_3.csv",index = False)